# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re

# download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from scipy.stats import hmean
from scipy.stats.mstats import gmean

In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterMessages.db')
df = pd.read_sql_table('disaster_msg_df', engine)
# X = df.message.values
# y = df.iloc[:,4:].values

In [5]:
df[~(df.isnull().any(axis=1))|(df.original.isnull())].shape, df[~(df.isnull().any(axis=1))].shape

((26345, 40), (10223, 40))

In [6]:
df = df[~(df.isnull().any(axis=1))|((df.original.isnull())&~(df.offer.isnull()))]

In [7]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df.genre.value_counts()

news      12978
direct    10825
social     2404
Name: genre, dtype: int64

In [9]:
df.shape

(26207, 40)

In [10]:
# drop all null values
#df = df.dropna()
df = df.dropna(subset=['original'])

In [11]:
df.shape

(10223, 40)

In [12]:
#df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [13]:
df.isnull().sum()

id                        0
message                   0
original                  0
genre                     0
related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10223 entries, 0 to 12416
Data columns (total 40 columns):
id                        10223 non-null int64
message                   10223 non-null object
original                  10223 non-null object
genre                     10223 non-null object
related                   10223 non-null float64
request                   10223 non-null float64
offer                     10223 non-null float64
aid_related               10223 non-null float64
medical_help              10223 non-null float64
medical_products          10223 non-null float64
search_and_rescue         10223 non-null float64
security                  10223 non-null float64
military                  10223 non-null float64
child_alone               10223 non-null float64
water                     10223 non-null float64
food                      10223 non-null float64
shelter                   10223 non-null float64
clothing                  10223 non-null float64
money         

In [15]:
X = df.message#.values
y = df.iloc[:,4:]#.values
categories = y.columns

### 2. Write a tokenization function to process your text data

In [17]:
def tokenize(text):
    # Identify and replace url with placeholder
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    
    # Remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9_]", " ", text)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [t for t in tokens if t not in stopwords.words("english")]
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    
    # iterate through each token and lemmatize, normalize case, and remove leading/trailing white space
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
    
#     #pass

In [18]:
for msg in X:
    tokens = tokenize(msg)
    #print(msg)
    #print(tokens, '\n')

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [19]:
#def my_pipeline():
    
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))])
    #return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [21]:
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [23]:
# #def display_results(y_test, y_pred):
# labels = np.unique(y_pred)
# confusion_mat = classification_report(y_test, y_pred, labels=labels)
# accuracy = (y_pred == y_test).mean()

# print("Labels:", labels)
# print("Confusion Matrix:\n", confusion_mat)
# print("Accuracy:", accuracy)

In [24]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: related

             precision    recall  f1-score   support

        0.0       0.37      0.26      0.31       864
        1.0       0.66      0.77      0.71      1664
        2.0       0.33      0.04      0.06        28

avg / total       0.56      0.59      0.57      2556

------------------------------------------------------

FEATURE: request

             precision    recall  f1-score   support

        0.0       0.67      0.85      0.75      1674
        1.0       0.40      0.19      0.26       882

avg / total       0.58      0.62      0.58      2556

------------------------------------------------------

FEATURE: offer

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2554
        1.0       0.00      0.00      0.00         2

avg / total       1.00      1.00      1.00      2556

------------------------------------------------------

FEATURE: aid_related

    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:
overall_accuracy = (y_pred == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 92.53% 



### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
# #def my_pipeline():
    
# pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
#                      ('tfidf', TfidfTransformer()),
#                      ('clf', MultiOutputClassifier(RandomForestClassifier()))])
#     #return pipeline

In [27]:
# pipeline = Pipeline([
#     ('features', FeatureUnion([

#         ('nlp_pipeline', Pipeline([
#             ('vect', CountVectorizer(tokenizer=tokenize)),
#             ('tfidf', TfidfTransformer())
#              ]),

#         ('txt_len', TextLengthExtractor())
#     ])),

#     ('clf', MultiOutputClassifier(RandomForestClassifier()))
#     ]))

In [28]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [29]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)), 
    'vect__max_df': (0.75, 1.0)
    #'clf__estimator__n_estimators':[50, 100]
    #'vect__max_features': (None, 5000), 
    #'tfidf__use_idf': (True, False) 
}
cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs = -1)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [30]:
# pipeline = Pipeline([
#     ('features', FeatureUnion([

#         ('nlp_pipeline', Pipeline([
#             ('vect', CountVectorizer(tokenizer=tokenize)),
#             ('tfidf', TfidfTransformer())
#         ])),

#         ('starting_verb', StartingVerbExtractor())
#     ])),

#     ('clf', MultiOutputClassifier(RandomForestClassifier()))
#     ])

In [31]:
# parameters = {
#     #'features__nlp_pipeline__vect__ngram_range': ((1, 1), (1, 2)) 
#     'features__nlp_pipeline__vect__max_df': (0.75, 1.0) 
#     #'features__nlp_pipeline__vect__max_features': (None, 5000), 
#     #'features__nlp_pipeline__tfidf__use_idf': (True, False) 
# }
# cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs = -1)
# cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [32]:
y_pred = cv.predict(X_test)

In [33]:
y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
for column in y_test.columns:
    print('------------------------------------------------------\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_pred_pd[column]))

------------------------------------------------------

FEATURE: related

             precision    recall  f1-score   support

        0.0       0.43      0.16      0.23       864
        1.0       0.66      0.89      0.76      1664
        2.0       0.50      0.04      0.07        28

avg / total       0.58      0.63      0.57      2556

------------------------------------------------------

FEATURE: request

             precision    recall  f1-score   support

        0.0       0.66      0.88      0.76      1674
        1.0       0.41      0.15      0.22       882

avg / total       0.58      0.63      0.57      2556

------------------------------------------------------

FEATURE: offer

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2554
        1.0       0.00      0.00      0.00         2

avg / total       1.00      1.00      1.00      2556

------------------------------------------------------

FEATURE: aid_related

    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
overall_accuracy = (y_pred == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 92.86% 



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [35]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion

In [36]:
def compute_text_length(data):
    return np.array([len(text) for text in data]).reshape(-1, 1)

In [37]:
pipeline = Pipeline([
    ('features', FeatureUnion([('text', Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                                                 ('tfidf', TfidfTransformer()),
                                                 ])),
                              ('length', Pipeline([('count', FunctionTransformer(compute_text_length, validate=False))]))]
                             )),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))])
#('clf', MultiOutputClassifier(AdaBoostClassifier()))
#('clf', MultiOutputClassifier(RandomForestClassifier()))])

In [38]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [39]:
overall_accuracy = (y_pred == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 92.88% 



In [40]:
parameters = {'features__text__vect__ngram_range': ((1, 1), (1, 2)), 
    'features__text__vect__max_df': (0.75, 1.0)
    #'clf__estimator__n_estimators':[50, 100]          
             }
    #'features__nlp_pipeline__vect__max_features': (None, 5000), 
    #'features__nlp_pipeline__tfidf__use_idf': (True, False) 

In [41]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text', 'features__length', 'features__text__memory', 'features__text__steps', 'features__text__vect', 'features__text__tfidf', 'features__text__vect__analyzer', 'features__text__vect__binary', 'features__text__vect__decode_error', 'features__text__vect__dtype', 'features__text__vect__encoding', 'features__text__vect__input', 'features__text__vect__lowercase', 'features__text__vect__max_df', 'features__text__vect__max_features', 'features__text__vect__min_df', 'features__text__vect__ngram_range', 'features__text__vect__preprocessor', 'features__text__vect__stop_words', 'features__text__vect__strip_accents', 'features__text__vect__token_pattern', 'features__text__vect__tokenizer', 'features__text__vect__vocabulary', 'features__text__tfidf__norm', 'features__text__tfidf__smooth_idf', 'features__text__tfidf__sublinear_tf', 'features__text__tfidf__us

In [42]:
cv = GridSearchCV(pipeline, parameters)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, ma...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'features__text__vect__ngram_range': ((1, 1), (1, 2)), 'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [43]:
y_pred = cv.predict(X_test)

In [44]:
overall_accuracy = (y_pred == y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 92.88% 



### 9. Export your model as a pickle file

In [2]:
import pickle

In [3]:
pickle.dump(cv, open('model.p', 'wb', encoding='UTF-8'))

NameError: name 'cv' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.